# Final Notebook - Advanced business analytics

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from matplotlib import cm
from PIL import Image
from IPython.display import Image
import os
import PIL
import glob
tqdm.pandas()
df = pd.read_csv("Data/articles_data.csv")
df=df.drop("Unnamed: 0",axis=1)
df=df.dropna()

## Introduciton


![alt text](Data/cover_pic.jpg "Title")

<a class="anchor" id="first-bullet"></a>
The impact social media platforms have had on the distribution of articles from news outlets has been significant. About half of U.S. adults (53%) say they get news from social media “often” or “sometimes,” and this use is spread out across a number of different sites, according to a Pew Research Center survey conducted Aug. 31-Sept. 7, 2020. Among 11 social media sites asked about as a regular source of news, Facebook sits at the top, with about a third (36%) of Americans getting news there regularly. But what articles a shown to which users?[1]

The facebook algorithm has long been a hot topic what news feed is shown to you. The specific details of how it works is not known, however there are supposedly three main ranking parameters.
* 1. Who posted it? Content by people or business affilated with you will be priotized.
* 2. The type of content, if you a more prone to click on videos, videos will be shown to you.
* 3. Interaction with the post, feed will prioritize posts with a lot of engagement, especially from people you interact with a lot.[2]

So when an article is shown to the user, a long process has been gone through to pick that exact one. But what determines whether the user will enteract with the article, is it the cover picture, title, news oulet or a combination of all?

This notebook will try to explore what is important for news articles on facebook, in order for people to enteract with them. This can be usefull for editors of newspaper to ensure that the articles they choose to share, are the one who will gain the most traction.

 We will try to built several models to predict the user engangment, with a special focus on article headlines, the cover picture or the news outlet. In order to achieve this several methods has been used: web-scraping, top-modelling, deep learning. The main data file for this project is obtained from https://www.kaggle.com/szymonjanowski/internet-articles-data-with-users-engagement, and webscrapes & API usuage has been carried out to supplement the data set.

[1] - https://www.pewresearch.org/journalism/2021/01/12/news-use-across-social-media-platforms-in-2020/ \
[2] - https://blog.hootsuite.com/facebook-algorithm/

## Data Sources

In this project three data sources are used.

- A data set containing 10437 rows representing unique articles from September 2019 to October 2019. The data set contain 15 columns where 11 of the are information about the article and the 4 of the are metrics on different engangement type with the article.